In [2]:
import numpy as np
import matplotlib.pyplot as plt
from build_FDTD import build_FDTD
from build_FDTD import build_crystal

C:\Program Files\Lumerical\v232\api\python\lumapi.py:895: SyntaxWarning: invalid escape sequence '\s'
  message = re.sub('^(Error:)\s(prompt line)\s[0-9]+:', '', str(rvals[2])).strip()


The goal is to create a photonic crystal for each individual mode exiting the multimodal fiber

In [3]:
fdtd = build_FDTD(lumerical_name= 'test', injection_angle=-20)

In [4]:
RING = {
    "inner_radius": 0.9384,  # [um]
    "outer_radius": 1.3432,  # [um]
    "depth": 0.070,  # [um]
    "matname": "etch"
}

In [5]:
fdtd.select("FDTD::ports::input_port")
fdtd.set("mode selection", "user select")
fdtd.set("selected mode numbers", 1)

In [6]:
# calculate grating pitch
lambda0 = 1.55  # [um]
ff = 0.5
n_etch = 1.536 # TE0
n_wg =  1.65433 # TE0
n_eff = n_wg*ff + n_etch*(1-ff)
n_clad = 1
theta = -20
# get pitch
pitch = lambda0/(n_eff - n_clad*np.sin(np.deg2rad(theta)))
print(pitch)

CIRCLE = {
    "radius": ((1-0.5)*pitch)/2,  # [um]
    "depth": 0.11,  # [um]
    "matname": "etch"
}

0.8001300264666662


In [7]:
crystal_args = {
    "dimension": 200,
    "crystal_constant": pitch,
    "scatterer": "CIRCLE",
    "scatterer_kwargs": CIRCLE,
    "matname": "etch",
}
build_crystal(fdtd, **crystal_args)